In [ ]:
import os
import time
import pickle
import numpy as np
import configparser

from os.path import join, abspath, dirname
from flask import Flask, request, jsonify, session
from flask_session import Session
import os
import cv2
import pickle
import numpy as np
import pandas as pd
import configparser
import matplotlib.pyplot as plt

from os.path import dirname, abspath, join
from tqdm import tqdm
from numpy.linalg import norm
from tensorflow.keras import Sequential, Model
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet_v2 import ResNet50V2,preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input


# base_path = dirname(abspath(__file__))



app = Flask(__name__)

@app.route('/predict', methods=["POST", "GET"])
def predict():
    base_path = os.getcwd()
    model_path = join(base_path, 'models')
    temp_path = join(base_path, 'temp')
    temp_file = join(temp_path, 'img_process.jpg')
    
    start_time = time.time()
    image_file = request.files["image"]
    image_bytes = image_file.read()
    with open(temp_file, 'wb')  as outfile:
        outfile.write(image_bytes)
    
    model = ResNet50V2(weights='imagenet',include_top=False,input_shape=(224,224,3))
    model.trainable = False

    model = Sequential([
        model,
        GlobalMaxPooling2D()
    ])
    
    feature_list = np.array(pickle.load(open('models/resnet50v2/resv2features.pkl', 'rb')))
    filenames = pickle.load(open('models/resnet50v2/resv2filenames.pkl', 'rb'))   
    
    print(filenames)
    # feature_list = np.array(pickle.load(open('models/vgg19/vgg19features.pkl', 'rb')))
    # filenames = pickle.load(open('models/vgg19/vgg19filepath.pkl', 'rb'))   
    
    # new_filenames = [f"{data.split('/')[-2:][0]}/{data.split('/')[-2:][1]}" for data in filenames]
    print(len(filenames))
    img = load_img(temp_file,target_size=(224,224))
    img_array = img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    neighbors = NearestNeighbors(n_neighbors=5,algorithm='brute',metric='cosine')
    neighbors.fit(feature_list)

    distances,indices = neighbors.kneighbors([normalized_result])

    img = cv2.imread(temp_file)
    # plt.imshow(img)
    # plt.title('query image')
    # plt.axis("off")
    # plt.show()

    pred = []
    for i in range(5):
        index = indices[0][i]
        distance = distances[0][i]
        # group = filenames[index].split('/')[-2:][0]#.split('.jpg')[0]
        name = filenames[index].split('.png')[0]
        # img_hehe = cv2.imread(f"tkquick-group/{group}/{name}.jpg")
        # plt.imshow(img_hehe)
        # plt.title(name)
        # plt.axis("off")
        # plt.show()'
        print(index, distance)
        pred.append(name)

    result = {
        "success": True,
        "pred": pred,
        "time":round(time.time() - start_time, 2)
    }
    result_json = jsonify(result)
    return result_json

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=6000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6000
 * Running on http://192.168.13.35:6000
Press CTRL+C to quit


['V-Belt B - 45 Bando.jpg', 'V-Belt B - 42 Bando.jpg', 'V-Belt B - 41 Bando.jpg', 'V-Belt B - 38 Bando.jpg', 'V-Belt B - 39 Bando.jpg', 'V-Belt B - 40 Bando.jpg', 'V-Belt B - 44 Bando.jpg', 'V-Belt B - 43 Bando.jpg', 'V-Belt B - 37 Bando.jpg', 'V-Belt B - 34 Bando.jpg', 'V-Belt B - 35 Bando.jpg', 'V-Belt B - 32 Bando.jpg', 'V-Belt B - 31 Bando.jpg', 'V-Belt B - 36 Bando.jpg', 'V-Belt B - 33 Bando.jpg', 'V-Belt B - 30 Bando.jpg', 'V-Belt B - 29 Bando.jpg', 'V-Belt A - 110 Bando.jpg', 'V-Belt A - 125 Bando.jpg', 'V-Belt B - 28 Bando.jpg', 'V-Belt A - 103 Bando.jpg', 'V-Belt A - 106 Bando.jpg', 'V-Belt A - 109 Bando.jpg', 'V-Belt A - 100 Bando.jpg', 'V-Belt A - 102 Bando.jpg', 'V-Belt A - 98 Bando.jpg', 'V-Belt A - 99 Bando.jpg', 'V-Belt A - 94 Bando.jpg', 'V-Belt A - 93 Bando.jpg', 'V-Belt A - 91 Bando.jpg', 'V-Belt A - 90 Bando.jpg', 'V-Belt A - 92 Bando.jpg', 'V-Belt A - 97 Bando.jpg', 'V-Belt A - 96 Bando.jpg', 'V-Belt A - 95 Bando.jpg', 'V-Belt A - 89 Bando.jpg', 'V-Belt A - 88 Bando

1/1 [==============================] - 1s 734ms/step


127.0.0.1 - - [30/Aug/2023 17:24:03] "POST /predict HTTP/1.1" 200 -


3368 0.59094906
3206 0.5909863
2733 0.5939492
4128 0.59487116
3227 0.59536934
